# 📘 Step 1: Load Raw Data

In [30]:
import pandas as pd

# Load the raw dataset
raw_df = pd.read_csv("/Users/yoey_danjuma/mobile-app-retention-analysis/data/raw_user_events.csv")
raw_df.head()

,user_id,install_date,event_date,event_type
0,U00001,2024-02-21,2024-03-04,app_open
1,U00001,2024-02-21,2024-02-28,app_open
2,U00001,2024-02-21,2024-03-14,app_open
3,U00002,2024-01-15,2024-01-28,app_open
4,U00003,2024-03-12,2024-03-30,app_open


# 🧹 Step 2: Data Cleaning & Transformation

In [31]:
## 2.1 Check for missing or duplicated data
print("Missing values:\n", raw_df.isnull().sum())
print("Duplicates:", raw_df.duplicated().sum())

Missing values:
 user_id         0
install_date    0
event_date      0
event_type      0
dtype: int64
Duplicates: 1365


In [32]:
## 2.2 Convert date columns to datetime
raw_df['install_date'] = pd.to_datetime(raw_df['install_date'])
raw_df['event_date'] = pd.to_datetime(raw_df['event_date'])

In [33]:
## 2.3 Remove invalid events (event before install)
raw_df = raw_df[raw_df['event_date'] >= raw_df['install_date']]

In [34]:
## 2.4 Remove any existing cohort_date columns to prevent confusion
raw_df = raw_df.drop(columns=[col for col in raw_df.columns if 'cohort_date' in col])

In [35]:
## 2.5 Recalculate cohort_date from install_date
raw_df['cohort_date'] = raw_df['install_date']

In [36]:
## 2.6 Calculate days_since_install
raw_df['days_since_install'] = (raw_df['event_date'] - raw_df['install_date']).dt.days

In [37]:
## 2.7 Remove duplicate user-day rows (based on days_since_install)
raw_df = raw_df.drop_duplicates(subset=['user_id', 'cohort_date', 'days_since_install'])

In [38]:
## 2.8 Sort by user and event date
cleaned_df = raw_df.sort_values(by=['user_id', 'event_date'])

In [39]:
## 2.9 Save cleaned data to CSV
cleaned_df.to_csv("/Users/yoey_danjuma/mobile-app-retention-analysis/cleaned_data.csv", index=False)

In [40]:
# Preview the cleaned data
cleaned_df.head()

,user_id,install_date,event_date,event_type,cohort_date,days_since_install
1,U00001,2024-02-21,2024-02-28,app_open,2024-02-21,7
0,U00001,2024-02-21,2024-03-04,app_open,2024-02-21,12
2,U00001,2024-02-21,2024-03-14,app_open,2024-02-21,22
3,U00002,2024-01-15,2024-01-28,app_open,2024-01-15,13
4,U00003,2024-03-12,2024-03-30,app_open,2024-03-12,18


In [41]:
# Step 3: Create Retention Table
cohort_data = cleaned_df.groupby(['cohort_date', 'days_since_install'])['user_id'].nunique().reset_index()

# Pivot the data to get the retention matrix
retention_matrix = cohort_data.pivot(index='cohort_date', columns='days_since_install', values='user_id')

# ✅ Correct cohort size: total users who installed that day
cohort_sizes = cleaned_df.groupby('cohort_date')['user_id'].nunique()

# Normalize the matrix
retention_rate = retention_matrix.divide(cohort_sizes, axis=0)

# Preview the final retention table
retention_rate.round(3)


days_since_install,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
cohort_date,,,,,,,,,,,,,,,,,,,,,
2024-01-01,0.065,0.131,0.103,0.075,0.112,0.065,0.084,0.103,0.131,0.037,...,0.084,0.037,0.093,0.121,0.131,0.093,0.131,0.103,0.112,0.056
2024-01-02,0.076,0.059,0.076,0.092,0.101,0.067,0.084,0.126,0.059,0.042,...,0.101,0.042,0.067,0.084,0.101,0.118,0.084,0.084,0.076,0.118
2024-01-03,0.102,0.093,0.083,0.102,0.120,0.111,0.120,0.083,0.120,0.130,...,0.093,0.130,0.102,0.157,0.176,0.093,0.083,0.074,0.093,0.074
2024-01-04,0.054,0.108,0.099,0.072,0.081,0.144,0.072,0.054,0.126,0.135,...,0.099,0.081,0.090,0.108,0.045,0.135,0.117,0.153,0.072,0.081
2024-01-05,0.106,0.128,0.074,0.074,0.106,0.138,0.074,0.064,0.106,0.128,...,0.096,0.074,0.149,0.117,0.064,0.128,0.074,0.074,0.128,0.085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-27,0.078,0.062,0.062,0.109,0.078,0.086,0.102,0.070,0.102,0.078,...,0.133,0.086,0.102,0.070,0.102,0.156,0.109,0.078,0.125,0.086
2024-03-28,0.132,0.114,0.070,0.088,0.114,0.105,0.070,0.035,0.114,0.070,...,0.096,0.096,0.088,0.088,0.053,0.079,0.088,0.149,0.070,0.167
2024-03-29,0.058,0.078,0.107,0.107,0.068,0.068,0.126,0.058,0.068,0.049,...,0.029,0.097,0.097,0.097,0.155,0.068,0.117,0.058,0.136,0.107
